<a href="https://colab.research.google.com/github/Kevinruy/Proyecto_Desgraduacion/blob/main/Proyecto__Desgraduacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install plotly==5.4 dash jupyter-dash

mkdir assets/
wget https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css -O assets/style.css
wget https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv -O web_data.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


mkdir: cannot create directory ‘assets/’: File exists
--2022-07-28 20:03:23--  https://gist.githubusercontent.com/escape-velocity-labs/17c3ba12aef542afe5055a859e0fbd98/raw/af9e2431fb69c76618aeec9940cd5fd6deab8b1f/style.css
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3793 (3.7K) [text/plain]
Saving to: ‘assets/style.css’

     0K ...                                                   100% 31.1M=0s

2022-07-28 20:03:23 (31.1 MB/s) - ‘assets/style.css’ saved [3793/3793]

--2022-07-28 20:03:23--  https://gist.githubusercontent.com/escape-velocity-labs/81b8225d72306541018bf868f3a23436/raw/c14fdac538abc45f8e19c4828bb015aa931c41c5/web_data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.1

In [2]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [3]:
app= JupyterDash(__name__)
app.title= """Análisis de Sitio Web"""

In [4]:
dataset= pd.read_csv('web_data.csv')

In [5]:
dataset.head()

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing


In [6]:
colors= ['#f2fffb', '#98ffe0', '#6df0c8', '#59dab3', '#31c194', '#25a27b', '#188463', '#11684d']


In [7]:
custom_theme= pio.templates["plotly_dark"]
custom_theme.layout.update({
    'paper_bgcolor': '#1f2630',
    'plot_bgcolor': '#1f2630',
    'colorway': colors,
    'colorscale_sequential': colors,
    'font': {
        'color':'#2cfec1'
    },
    'margin':{
        't':75,
        'r':50,
        'b': 100,
        'l':75
    }
})
pio.templates.default = custom_theme

#### Dashboard structure

#### Dashboard header

In [8]:
header = html.Div(
    id= 'header',
    children= [
               html.H4('Análisis de trafico en un sitio web'),
               html.P(
                   id='description',
                   children= """Los datos de este sitio web es parte de el archivo web_data.csv del usuario escape-velocity-labs (https://github.com/escape-velocity-labs)\n.
                   Analarizaremos una serie de tiempo para las visitas al sitio web\n.
                   Crearemos un tunel de ventas para determinar desde que sistemas operativos y paises nos visitan\n.
                   Crearemos un gráfico el cual nos muestre la distribución de visitas\n.
                   Los periodos clave de actividad dentro de nuestro sitio web\n
                   y por último el numero de visitas por pais"""
               )
    ]
)

#### Evolution of website visits

In [9]:
visits_per_day= dataset.value_counts('date', sort=False)
visits_per_day.head()

date
2021-10-01    307
2021-10-02    308
2021-10-03    342
2021-10-04    325
2021-10-05    331
dtype: int64

In [10]:
trend_title = html.P(className='chart-header', children='Visitas al sitio web por día')

In [11]:
labels = {'date':'Date', 'value':'Value'}
trend_figure = px.line(visits_per_day, labels=labels, markers=True)
trend_figure.layout.update(showlegend=False)
trend_graph= dcc.Graph(figure=trend_figure, className='graph')

#### First row

In [12]:
first_row = html.Div(
    className= 'graph-row',
    children=[
              html.Div(
                  className='graph-container',
                  children=[
                            trend_title,
                            trend_graph
                  ]
              )
    ]
)

#### Sales funnel

In [13]:
funnel_title = html.P(className='chart-header', children= 'Tunel de ventas')

In [14]:
@app.callback(Output('funnel-graph', 'figure'), [Input('os-dropdown', 'value'), Input('country-dropdown', 'value')])
def update_funnel_graph (os,country):
  subset= dataset
  if os:
    subset = subset.query(f"os== '{os}'") #La f hará que los valores dentro de los corchetes tomen el valor de lo que llamemos
  #Las comillas nos ayudan a que pandas lo interprete como un string
  if country:
    subset = subset.query(f"country == '{country}'")
  visits_per_page= subset.value_counts('page')
  fig = px.funnel(visits_per_page, color = visits_per_page)
  return fig

In [15]:
os_dropdown= dcc.Dropdown(
    id='os-dropdown',
    placeholder='Filtro por sistema operativo',
    options=[
             {'label':'Windows', 'value':'Windows'},
             {'label':'Android', 'value':'Android'},
             {'label':'Mac', 'value':'Mac'},
             {'label':'iOS', 'value':'iOS'},
             {'label':'Linux', 'value':'Linux'}
    ]
)

country_dropdown = dcc.Dropdown(
    id='country-dropdown',
    placeholder='Filtro por país',
    options=[
             {'label':'Estados unidos', 'value': 'United States'},
             {'label':'Canada', 'value': 'Canada'},
             {'label':'Reino Unido', 'value': 'United Kingdom'},
             {'label':'Australia', 'value': 'Australia'},
             {'label':'Nueva Zelanda', 'value': 'New Zealand'},
             {'label':'Brasil', 'value': 'Brazil'},
             {'label':'Rusia', 'value': 'Russia'},
             {'label':'Alemania', 'value': 'Germany'},
             {'label':'México', 'value': 'Mexico'},
    ]
)

In [16]:
funnel_controls= html.Div(
    className='controler-row',
    children=[
              os_dropdown,
              country_dropdown
    ]
)

In [17]:
funnel_graph= dcc.Graph(id='funnel-graph', className='graph')

#### Distribution of visits by attribute

In [18]:
pie_title = html.P(className='chart-header', children='Distribución de visitas')

In [19]:
@app.callback(Output('pie-graph','figure'), [Input('radio-buttons', 'value')])
def update_pie_chart(value):
  fig= px.pie(dataset, names=value)
  return fig

In [20]:
radio_buttons= dcc.RadioItems(
    id='radio-buttons',
    options=[
             {'label':'Sistema operativo', 'value':'os'},
             {'label':'País', 'value':'country'}
    ],
    value='os',
    labelStyle={'display': 'inline-block'} #Propiedad del estilo que te dice donde colocar los botones
)

In [21]:
pie_controls = html.Div(
    className='controler-row',
    children=[
              radio_buttons
    ]
)

In [22]:
pie_graph= dcc.Graph(id='pie-graph')

#### Second row

In [23]:
second_row= html.Div(
    className= 'graph-row',
    children=[
              html.Div(
                  className='graph-container',
                  children=[
                            funnel_title,
                            funnel_controls,
                            funnel_graph
                  ]
              ),
              html.Div(
                  className='graph-container',
                  children=[
                            pie_title,
                            pie_controls,
                            pie_graph
                  ]
              )
    ]
)

#### Visits by date and time

In [24]:
dataset.head()

,date,time,weekday,os,country,page
0,2021-10-01,00:00:00,Friday,Windows,United States,Detail
1,2021-10-01,00:00:00,Friday,Mac,United Kingdom,Shopping Cart
2,2021-10-01,00:00:00,Friday,Linux,Brazil,Detail
3,2021-10-01,00:00:00,Friday,Windows,Canada,Detail
4,2021-10-01,00:00:00,Friday,Linux,Germany,Landing


In [25]:
visits_day_time = dataset.groupby(['weekday', 'time']).size().reset_index(name='counts')
visits_day_time.head()

,weekday,time,counts
0,Friday,00:00:00,44
1,Friday,01:00:00,48
2,Friday,02:00:00,53
3,Friday,03:00:00,45
4,Friday,04:00:00,55


In [26]:
visits_day_time = visits_day_time.pivot_table(values='counts', index='time', columns='weekday')
visits_day_time.head()

weekday,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
time,,,,,,,
00:00:00,44,38,42,39,47,41,43
01:00:00,48,46,53,48,46,53,45
02:00:00,53,44,58,46,51,44,37
03:00:00,45,35,50,54,35,39,66
04:00:00,55,46,43,51,41,45,46


In [27]:
heatmap_title = html.P(className='chart-header', children='Periodos clave de actividad')

In [28]:
heatmap_figure= go.Figure(
    data=go.Heatmap(
        x=visits_day_time.columns.tolist(),
        y=visits_day_time.index.tolist(),
        z=visits_day_time.values.tolist(),
        autocolorscale=True,
        xgap=4,
        ygap=1
    )
)
layout= heatmap_figure.layout.yaxis.update(autorange='reversed', tickvals=[0,6,12,18], ticktext=['12 AM', '6 AM', '12 PM', '6 PM'])

In [29]:
heatmap_graph= dcc.Graph(figure=heatmap_figure, className='graph')

#### Distribution by country

In [30]:
country_title = html.P(className= 'chart-header', children = 'Visitas por país')

In [31]:
country_figure= px.histogram(dataset, x='country', color='country')
layout= country_figure.layout.update(showlegend= False)

In [32]:
country_graph= dcc.Graph(figure=country_figure, className='graph')

In [33]:
third_row = html.Div(
    className= 'graph-row',
    children=[
              html.Div(
                  className='graph-container',
                  children=[
                            heatmap_title,
                            heatmap_graph
                  ]
              ),
              html.Div(
                  className='graph-container',
                  children=[
                            country_title,
                            country_graph
                  ]
              )
    ]
)

#### Create the dashboard

In [34]:
app.layout= html.Div(
    id='root',
    children=[
             header,
              first_row,
              second_row,
              third_row
    ]
)

In [35]:
app.run_server(debug=True, mode='external')

Dash app running on:


<IPython.core.display.Javascript object>